In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec 10 16:46:48 2019

@author: XE86670
"""

import csv
import pprint
import xlrd

import plotly as plotly
import plotly.graph_objs as go
import numpy as np
from ipywidgets import widgets, Layout, Box, Button

import pandas as pd
from IPython.display import display
import plotly.graph_objs as go






class Aplicacion:
    
    def __init__(self):
        
        
        #Lee los archivos
        self.lectura_archivos()
        
        self.calcular_percentil()
        
        df = pd.DataFrame(self.data)
        
     
        
        #Las siguientes lineas, incluido el bucle, las uso para transformar la lista de valores de patrimonio al formato correcto 
        #En este caso un array para poder modificar el tamaño de los puntos de la gráfica
        aux = df.iloc[0].values
        
   
        isins = df.columns.values
        
        
        
        pat_array_size = []
      
        for x in aux:
            
           
            
            pat_array_size.append(x*0.09)
            
        
        self.fig = go.Scatter( y=df.iloc[9], mode='markers', marker=dict(size=pat_array_size),
                              hovertext=df.iloc[1]+'<br>'+isins+'<br>', name="Posiciones")
                             #overinfo=["y",df.iloc[1],isins] )
                             
       
        
        
        self.fig_widget = go.FigureWidget(data=self.fig, layout=go.Layout(barmode='overlay', title_text='Posiciones', xaxis_title="Coste"))
        
        self.fig_widget.add_scatter(x=[0,800], y=[1,1], mode='lines', name="Mediana")
        
        self.fig_widget.add_scatter(x=[0,800], y=[self.p_ochenta, self.p_ochenta], mode='lines', name='Percentil 80')
        
        
        self.fig_widget.add_scatter(x=[0,800], y=[self.p_veinte, self.p_veinte], mode='lines', name='Percentil 20')
        
        self.w_grupo = widgets.SelectMultiple(
            description='Grupo',
            style={'description_width': 'initial'},
            options=['', 'ETF', 'FONDO']
        )
        
        self.w_geografia = widgets.SelectMultiple(
            description='Geografía',
            style={'description_width': 'initial'},
            options=['','EUROPA', 'MEXICO', 'PERU', 'ARGENTINA', 'B.P.DISCRECIONAL'],
           
        )
        
        self.w_qf1 = widgets.SelectMultiple(
            description='QF CATEGORY L1',
            style={'description_width': 'initial'},
            options=['', 'MIXTOS', 'RENTA FIJA', 'RENTA VARIABLE', 'ALTERNATIVOS', 'MUTUAL FUND'],
            
        )
        
        self.w_qf2 = widgets.SelectMultiple(
            description='QF CATEGORY L2',
            style={'description_width': 'initial'},
            options=['','AGGREGATE', 'COMMODITIES', 'CREDITO', 'EMERGENTE', 'EUROPA', 'GLOBAL', 'GOBIERNOS', 'HIGH YIELD', 'JAPON', 'MONETARIO', 'RETORNO ABSOLUTO', 'SECTORES', 'SMART BETA', 'TEMATICOS', 'USA'],
            
        )
        self.w_qf3 = widgets.SelectMultiple(
            description='QF CATEGORY L3',
            style={'description_width': 'initial'},
            options=['','ASIA EX JAPON', 'CORE','ENERGIA','EURO','FINANZAS','GLOBAL','GROWTH','HARD CURRENCY','INFRAESTRUCTURAS','LATAM','LOCAL CURRENCY','MATERIALES PRECIOSOS','MONOPAIS','OTROS', 'PANEUROPEO','PERFIL ALTO','PERFIL BAJO','PERFIL MEDIO','PERFIL MUY ALTO','REITS','US DOLAR','USA','VALUE'],
            
        )
        self.w_qf4 = widgets.SelectMultiple(
            description='QF CATEGORY L4',
            style={'description_width': 'initial'},
            options=['','BRASIL','CHILE','CORE','CORTO','CTA','EEUU','EQUITY MARKET NEUTRAL','GLOBAL','GLOBAL MACRO','GOBIERNOS','LARGO','LONG SHORT EQUITY','MULTI ESTRATEGIA DE RENTA FIJA Y DIVISAS','PREFERENTES','USA' ],
            
        )
        
        b_filtrar = Button(description='Filtrar', layout=Layout(width='25%', height='60px'))
        b_filtrar.align_items='center'
        self.out = widgets.Output()
        b_filtrar.on_click(self.on_button_clicked)
        
        
        self.container = widgets.HBox(children=[self.w_grupo, self.w_geografia, self.w_qf1])
        self.container2= widgets.HBox(children=[self.w_qf2, self.w_qf3, self.w_qf4])
       
        
        self.final = widgets.VBox([self.fig_widget, self.container, self.container2, b_filtrar])
        display(self.final)
        
        
    #Función de botón "Filtrar"  
    def on_button_clicked(self, out):
            
        
        grupo = self.w_grupo.value
        geografia = self.w_geografia.value
        
        qf1 = self.w_qf1.value
        qf2 = self.w_qf2.value
        qf3 = self.w_qf3.value
        qf4 = self.w_qf4.value
        
        
        self.aplicar_filtros(geografia, grupo, qf1,qf2,qf3,qf4)
        
  
    def calcular_percentil(self): 
        N = len(self.array_coste)
        
        if(N>0): 

            # Percentil 80
            ochenta = (N * 80)/100
            self.p_ochenta = self.array_coste[int(ochenta)]

            # Percentil 20
            veinte = (N * 20)/100
            self.p_veinte = self.array_coste[int(veinte)]

            print(self.p_ochenta)
            print(self.p_veinte)

    def actualizar_array_coste(self): 
        
        self.array_coste = np.array([]) 
        for key in list(self.filtrado): 
            aux = self.filtrado[key]
    
            self.array_coste = np.append(self.array_coste, aux[9])
                
        
        
        self.array_coste.sort()
        
    #Función que aplica los filtros    
    def aplicar_filtros(self, geo, grupo, L1,L2,L3,L4):
        
        #Diccionario donde se guardarán los datos que cohinciden con el filtro
        self.filtrado = {}
        
        
        f_grupo = list(grupo)
       
        f_geo = list(geo)
        
        
        f_l1 = list(L1)
        
        
        f_l2 = list(L2)

       
        f_l3 = list(L3)
       
        
        f_l4 = list(L4)
      
       
        
        try: 
            f_grupo.remove('')
            f_geo.remove('')
            f_l1.remove('')
            f_l2.remove('')
            f_l3.remove('')
            f_l4.remove('')
        except: 
            print('No en blanco')
        
        
        #Para cada uno de los fondos compruebo si cumplen las condiciones del filtro
        for item in self.data: 
            
         
            #Almaceno en una variable auxiliar el fondo para poder compararlo con el filtro
            aux = self.data[item]
           
            #Correcto controlará que el fondo cumple con los requisitos del filtro, en caso de 
            #no cumplir dichos requisitos cambiará a false y será descartado
            correcto = True
            #Compruebo que cada elemento del filtro coincide con el fondo
            
            if len(f_grupo)>0: 
                #Si el filtro no coincide el fondo es descartado y se pasa a comprobar el siguiente
                if aux[2] not in f_grupo: 
                    correcto = False
                    
                    

            
            if len(f_geo)>0:
                #Si el filtro no coincide el fondo es descartado y se pasa a comprobar el siguiente
                if aux[3] not in f_geo: 
                    correcto = False
                   

                    
            if len(f_l1)>0:
                #Si el filtro no coincide el fondo es descartado y se pasa a comprobar el siguiente
                if aux[4] not in f_l1: 
                    correcto = False
              

            
            
            if len(f_l2)>0:
                #Si el filtro no coincide el fondo es descartado y se pasa a comprobar el siguiente
                if aux[5] not in f_l2: 
                    correcto = False
                   
            
            
             
            if len(f_l3)>0:
                #Si el filtro no coincide el fondo es descartado y se pasa a comprobar el siguiente
                if aux[6] not in f_l3: 
                    correcto = False
                    
           
            
            if len(f_l4)>0:
                #Si el filtro no coincide el fondo es descartado y se pasa a comprobar el siguiente
                if aux[7] not in f_l4: 
                    correcto = False

          
                
                    
            #Si cumple todos los requisitos del filtro el fondo es guardado en el diccionario filtrados
            if correcto:  
             
                self.filtrado[item] = aux
            
                
              
          
        self.actualizar_grafica(self.filtrado)    
                
    #Función de botón "Cerrar"  
    def cerrar(self):
        self.root.quit()     
        self.root.destroy()
        
    #Función que dibuja la gráfica con los parámetros indicados
    def actualizar_grafica(self, info):
           
        df = pd.DataFrame(info)
        
        self.actualizar_array_coste()
        self.calcular_percentil()
        
        if not df.empty:
           

          
            aux = df.iloc[0].values



            isins = df.columns.values


            pat_array_size = []

            for x in aux:
                pat_array_size.append(x*0.09)



           # self.fig = go.Scatter( x=df.iloc[0], y = df.iloc[1], mode='markers')
            with self.fig_widget.batch_update():
                self.fig_widget.data[0].y = df.iloc[9]
                self.fig_widget.data[0].hovertext= df.iloc[1]+'<br>'+isins
                self.fig_widget.data[0].marker.size=pat_array_size
                
                self.fig_widget.data[2].y = [self.p_ochenta, self.p_ochenta]
                
                self.fig_widget.data[3].y = [self.p_veinte, self.p_veinte]
                
                #self.fig_widget.add_scatter(x=[0,800], y=[self.p_ochenta, self.p_ochenta], mode='lines', name='Percentil 80')
                #self.fig_widget.add_scatter(x=[0,800], y=[self.p_veinte, self.p_veinte], mode='lines', name='Percentil 20')
                #pprint.pprint(self.fig_widget.data[0])

    #Función de lectura de archivos
    def lectura_archivos(self):
        #Guardo en sheet la información de la hoja "Posiciones"
        self.book = xlrd.open_workbook(r'C:\Users\XE86670\FRAN\Informe_Fondos&ETFs.xlsx')
        self.sheet = self.book.sheet_by_name('Posiciones')
        print(self.sheet.nrows)
        
        # Diccionario->  clave = isin_clase
        # valores = Patrimonio, nombre cartera, grupo, geografía
        self.data = {}
        self.backup = {}
        

        
        for i in range(self.sheet.nrows): 
            
            row = self.sheet.row_values(i)
            isin_clase = row[9]
          
            
            # Evito procesar las primeras lineas vacías y el encabezado
            if isin_clase != "" and isin_clase != "ISIN clase": 
                
                #Si el isin de la clase ya existe, actualizo el coste
                if isin_clase in self.data:
                    #Extraigo los datos existentes para ese ISIN dentro del diccionario
                    aux = self.data[isin_clase]

                      
                    #No proceso la cabecera
                    if isinstance(row[19],float):
                        coste = aux[0] + row[19]                            
                   
                        
                #Si el ISIN aun no existe en el diccionario no es necesario actualizar por lo tanto se incluye una nueva entrada          
                else: 
                    
                    
                    #No proceso la cabecera
                     if isinstance(row[19],float):
                        coste = row[19]
                   
                 
                nombre = row[5]
                geografia = row[2]
                grupo = row[3]
                    
              

                #QF Categori L1
                qf1 = row[27]
                if qf1 == "": 
                    qf1 = "no_qf1"
                    
                #QF Categori L2
                qf2 = row[28]
                if qf2 == "": 
                    qf2 = "no_qf2"

                #QF Categori L3    
                qf3 = row[29]
                if qf3 == "": 
                    qf3 = "no_qf3"
                    
                #QF Categori L4    
                qf4 = row[30]
                if qf4 == "": 
                    qf4 = "no_qf4"

                
                
                
                
                self.data[isin_clase] = [coste,nombre, grupo, geografia, qf1, qf2,qf3, qf4]
          
                
            
        
     
        with open(r'C:\Users\XE86670\FRAN\fichero_QF_20190918.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=';')
            for row in csv_reader: 
                isin_share_class =row[0]
               
                if isin_share_class in self.data:
                    # PRECIO
                    self.data[isin_share_class].append(row[110])
                    # TOTAL_COMISSION
                    self.data[isin_share_class].append(row[13])    
                    # ONGOING_CHARGE
                    self.data[isin_share_class].append(row[14])
                    # BP_TOTAL_REBATE
                    self.data[isin_share_class].append(row[23])
                    # PERFORMANCE_FEE
                    self.data[isin_share_class].append(row[54])
                   
                
               
        self.array_coste = np.array([]) 
        for key in list(self.data): 
            aux = self.data[key]
           
            if len(aux)<9: 
                self.backup[key] = aux
                del self.data[key]
            else: 
                #Lo uso para calcular percentiles
                self.array_coste = np.append(self.array_coste, aux[9])
                
        
        
        self.array_coste.sort()
      
        
        
       
     
    
        
aplicacion1=Aplicacion()

11542
1.25
0.5


    'data': [{'hovertext': array(['ORAFLA 2000 SICAV<br>FR0011307065<br>', 'OLIV…